# TODO:

* Factor this
* Put it into the real python code so we can quiz people about battle outcomes (which need actual damage done)
* This code disagrees with pvpivs for the stat product, and it's because of floating point differences. The relative rankings will be the same.

In [22]:
import imp
import pvprankings
import pandas as pd
imp.reload(pvprankings)
df = pd.read_csv('poke_genie_export.csv')

In [23]:
pvprankings.display_rs_info(df,'Dunsparce')

# Dunsparces you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/dunsparce-pvp-iv-deep-dive)!

Check out the videos: [here](https://www.youtube.com/watch?v=ZiZEbDqMur0)!

Extra Tips: Cares ONLY about bulk, not about Atk at all

## Great League

### Basic Bulk

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### Premium Bulk

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


## Ultra League

In [24]:
pvprankings.display_rs_info(df,'Lickitung')

# Lickitungs you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/lickitung-pvp-iv-deep-dive)!

Check out the videos: [here]()!

Extra Tips: General Good is minimum bulk for "good" lickitung.
Atk focus: Potentiates Registeel and Shadow Walrein Breakpoints without giving up too much bulk, 126.2 Def, 126.58 Def, and 184 HP are notable stat checks, Priority Recommendation 126.2 Def=184 HP>185+ HP=126.58 Def
Budget Boi: This budget list is for players who’ve already built a Lickitung and are wondering if they should build twice (or for players looking to save ~94 XL Candy), If you meet at least 125.1 Def and 181 HP, building a better Lickitung might not be worth it compared to other projects

## Great League

...

### Atk Focus (97.7 Atk, 125.94 def, 183 hp)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### Budget Bois (97 Atk, 125.1 def, 181 hp)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
3,1147,1495,49.0,98.565399,125.294999,183,10.0,13.0,13.0,2260004


## Ultra League

In [14]:
pvprankings.display_rs_info(df,'Stunfisk (Galarian)')

# Stunfisk (Galarian)s you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/galarian-stunfisk-pvp-iv-deep-dive)!

Check out the videos: [here](https://www.youtube.com/watch?v=iUja0_EjGnc)!

Extra Tips: For UL, realy do want 15/15/14

## Great League

...

### bulk mirror slayer (101.79 atk, 124.75/172)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### Minimum bulk (124.75 def 172 hp)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
15,1472,1500,26.5,100.401996,127.909392,174,2.0,15.0,14.0,2234570
10,326,1495,26.5,101.089681,125.846337,173,3.0,12.0,13.0,2200865


## Ultra League

In [3]:
pvprankings.display_rs_info(df,'Deoxys (Defense)')

# Deoxys (Defense)s you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/deoxys-defense-pvp-iv-deep-dive-analysis)!

Check out the videos: [here](https://www.youtube.com/watch?v=p-UrAQDrvTQ)!

Extra Tips: For GL, the SwagMan wants the rank 67>77>22 (15/14/10 > 15/10/12 > 14/12/13) and then bulk 1 > 3 > 9 (10/15/13 > 10/10/15 > 11/13/12) then the mirror slayer rank 17 (13/11/15). For that first range, the 15/13/10 is my best so far. 
 The 12/13/15 I have is the UL rank 1. But also my best GL mirror slayer. For UL I decide between that rank 1 and my rank 150 15/13/12 and rank 146 15/11/12 which hit the table breakpoints.

## Great League

...

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### Sme rando Umb, TF, Regi

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
2,1293,2499,48.0,132.017698,283.962597,126,15.0,12.0,15.0,4723499


### Just atk breakpoint

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
43,1256,2500,50.5,132.319598,288.237597,123,13.0,12.0,10.0,4691156
0,1279,2500,49.0,132.812698,284.837297,124,15.0,11.0,12.0,4690921
18,1282,2492,48.5,132.415198,285.650397,124,15.0,13.0,12.0,4690232
35,1270,2498,49.5,133.210198,285.689797,123,15.0,11.0,10.0,4680985
31,1602,2491,48.5,132.415198,287.315997,123,15.0,15.0,11.0,4679535
38,1262,2497,50.0,132.767398,286.542297,123,14.0,11.0,10.0,4679347
57,1285,2497,48.5,132.415198,284.817597,124,15.0,12.0,13.0,4676558
8,1586,2496,49.5,132.372398,284.851997,124,14.0,10.0,12.0,4675611
61,1277,2498,49.0,132.812698,288.178497,122,15.0,15.0,10.0,4669399
29,1264,2486,49.5,132.372398,286.527597,123,14.0,12.0,10.0,4665186


In [3]:
pvprankings.display_rs_info(df,'Linoone (Galarian)')

# Linoone (Galarian)s you have, according to the Swag Man

Check out the article [here](https://twitter.com/SwgTips/status/1558892455017185280)!

Check out the videos: [here]()!

Extra Tips: Still bad right now.

## Great League

...

### SwagAtk FS Awak

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
94,181,1494,40.5,112.578163,113.370967,155,0.0,15.0,10.0,1978279
62,217,1493,38.5,114.141348,111.795978,153,4.0,15.0,10.0,1952363
2,383,1491,39.5,114.971147,111.821252,150,4.0,14.0,5.0,1928432
1,315,1492,38.5,114.923138,111.795978,150,5.0,15.0,7.0,1927191
0,339,1500,38.0,116.839912,111.387383,147,8.0,15.0,4.0,1913130


### Best

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
62,217,1493,38.5,114.141348,111.795978,153,4.0,15.0,10.0,1952363
2,383,1491,39.5,114.971147,111.821252,150,4.0,14.0,5.0,1928432
1,315,1492,38.5,114.923138,111.795978,150,5.0,15.0,7.0,1927191
0,339,1500,38.0,116.839912,111.387383,147,8.0,15.0,4.0,1913130


## Ultra League

In [131]:
pvprankings.display_rs_info(df,'Obstagoon')

# Obstagoons you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/obstagoon-pvp-iv-deep-dive)!

Check out the videos: [here](https://www.youtube.com/watch?v=XEygOnJDnlY)!

Extra Tips: You want three of each league, one with obstruct and one without, and one with high bulk. Also, this does not filter for galarian zig/lin; you have to do that yourself.
 For UL you really want 148+ attack, then 172 HP

## Great League

...

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,98,2498,40.0,149.366700,158.060000,177,9.0,6.0,12.0,4178775
123,2496,2496,37.0,149.224995,161.595979,173,13.0,15.0,12.0,4171749
10,389,2497,41.5,149.188603,160.357803,174,7.0,7.0,7.0,4162698
122,1117,2495,37.0,149.224995,159.276419,175,13.0,12.0,15.0,4159403
27,207,2500,41.0,149.516402,159.855302,174,8.0,7.0,8.0,4158772
98,356,2500,43.0,149.785800,159.449400,174,6.0,4.0,5.0,4155686
108,150,2496,42.5,149.320799,158.954399,175,6.0,4.0,7.0,4153659
113,346,2490,42.5,149.320799,158.954399,175,6.0,4.0,6.0,4153659
24,411,2495,37.5,149.780538,158.317252,175,13.0,10.0,14.0,4149747
97,89,2489,38.5,149.321900,158.703381,175,11.0,9.0,12.0,4147130


### Bulk Focus

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
94,181,2498,42.5,144.503999,167.785199,178,0.0,15.0,10.0,4315722


### All

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
94,181,2498,42.5,144.503999,167.785199,178,0.0,15.0,10.0,4315722
95,17,2486,42.0,144.054000,164.061500,181,0.0,11.0,15.0,4277702
59,217,2491,40.5,145.875930,165.696029,176,4.0,15.0,10.0,4254106
120,1083,2499,41.0,146.335202,163.036502,178,4.0,11.0,13.0,4246720
33,187,2495,41.5,146.795203,159.560003,181,4.0,6.0,15.0,4239498
...,...,...,...,...,...,...,...,...,...,...
68,304,2483,37.0,150.771367,154.637300,175,15.0,6.0,15.0,4080103
81,73,2492,40.0,151.737600,158.060000,170,12.0,6.0,4.0,4077219
2,79,2487,37.5,151.332668,157.541187,171,15.0,9.0,9.0,4076832
110,358,2488,37.5,151.332668,159.093317,169,15.0,11.0,7.0,4068846


In [128]:
pvprankings.display_rs_info(df,'Deoxys (Defense)')

# Deoxys (Defense)s you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/deoxys-defense-pvp-iv-deep-dive-analysis)!

Check out the videos: [here](https://www.youtube.com/watch?v=p-UrAQDrvTQ)!

## Great League

### Best you can get are here, but watch the actual video

...

## Ultra League

### BB umbreon

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
7,1274,2491,49.0,132.812698,286.507897,122,15.0,13.0,10.0,4642330


### Registeel and TF

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [2]:
pvprankings.display_rs_info(df,'Marowak (Alolan)')

# Marowak (Alolan)s you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/alolan-marowak-pvp-iv-deep-dive)!

Check out the videos: [here](https://www.youtube.com/watch?v=jMwFqbkVlpU)!

Extra Tips: Right now, this is looking at everything, so you have to check for shadow explicitly. Go for mid range attack if hit the bulk.

## Great League

...

### All

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
14,191,1496,34.5,109.242773,146.415661,128,0.0,7.0,14.0,2047341
6,366,1496,34.5,109.242773,146.415661,128,0.0,7.0,14.0,2047341
15,194,1491,33.5,108.392988,149.793088,126,0.0,13.0,13.0,2045801
5,370,1491,33.5,108.392988,149.793088,126,0.0,13.0,13.0,2045801
7,364,1499,35.0,110.426757,152.312768,121,1.0,14.0,4.0,2035148
13,191,1499,35.0,110.426757,152.312768,121,1.0,14.0,4.0,2035148
2,391,1499,30.5,112.415374,142.539756,124,9.0,8.0,15.0,1986933
11,211,1499,30.5,112.415374,142.539756,124,9.0,8.0,15.0,1986933
12,218,1493,29.5,112.464220,142.938396,123,11.0,11.0,15.0,1977280
1,399,1493,29.5,112.464220,142.938396,123,11.0,11.0,15.0,1977280


### Best

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


## Ultra League

# pvprankings.display_rs_info(df,'Araquanid')

In [23]:
pvprankings.display_rs_info(df,'Golem (Alolan)')

# Golem (Alolan)s you have, according to the Swag Man

Check out the article [here]()!

Check out the videos: [here]()!

Extra Tips: 

## Great League

...

### Min

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
49,282,1500,19.5,124.466348,123.876461,117,0.0,12.0,9.0,1803958
27,622,1498,19.0,124.025406,122.860848,118,2.0,13.0,14.0,1798068
15,184,1497,19.5,124.466348,123.286573,117,0.0,11.0,9.0,1795368
12,86,1494,19.0,124.025406,122.278569,118,2.0,12.0,14.0,1789546
79,83,1500,19.5,125.056236,123.286573,116,1.0,11.0,8.0,1788459
25,205,1497,19.0,125.189964,119.949453,119,4.0,8.0,15.0,1786959
5,449,1493,18.5,124.681502,122.383225,117,6.0,15.0,14.0,1785294
3,431,1494,19.0,125.189964,121.696290,117,4.0,11.0,11.0,1782513
62,51,1487,19.0,124.025406,120.531732,119,2.0,9.0,15.0,1778930
57,522,1499,18.5,125.830640,121.808656,116,8.0,14.0,13.0,1777962


### Best

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
44,1184,1500,19.0,126.354521,122.860848,114,6.0,13.0,7.0,1769738
0,367,1497,19.0,126.354521,119.367175,117,6.0,7.0,12.0,1764662
26,822,1494,20.0,126.051400,120.674800,116,0.0,4.0,5.0,1764502
35,87,1499,19.0,126.936800,122.860848,113,7.0,13.0,5.0,1762298
72,431,1493,19.0,126.354521,119.949453,116,6.0,8.0,10.0,1758114
67,398,1494,19.0,126.936800,124.025406,111,7.0,15.0,2.0,1747516
84,366,1490,20.0,126.648800,120.674800,114,1.0,4.0,2.0,1742298
76,577,1492,18.0,127.519763,118.451691,115,14.0,11.0,14.0,1737067
51,235,1490,18.5,127.554348,120.084949,113,11.0,11.0,8.0,1730861
81,434,1499,19.0,129.848195,119.949453,111,12.0,8.0,1.0,1728849


## Ultra League

In [8]:
pvprankings.display_rs_info(df,'Araquanid')

# Araquanids you have, according to the Swag Man

Check out the article [here]()!

Check out the videos: [here](https://www.youtube.com/watch?v=dQKFwYr9tQY)!

Extra Tips: In general, HP > def, Atk > b/c walrus.

## Great League

...

### HP sable max

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
4,144,1499,36.0,96.692043,174.199158,138,0.0,8.0,11.0,2324426
9,10,1499,37.0,97.421499,169.327844,139,0.0,0.0,12.0,2292967


### General

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
1,117,1497,33.5,99.360239,168.611315,134,6.0,5.0,10.0,2244936


## Ultra League

In [6]:
pvprankings.display_rs_info(df,'Azumarill')

# Azumarills you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/azumarill-pvp-iv-deep-dive)!

Check out the videos: [here](https://www.youtube.com/watch?v=iYaqdQG0Ic8)!

Extra Tips: Old non-XL article: https://gamepress.gg/pokemongo/azumarill-great-league-pvp-iv-deep-dive .. you can make up or lower def/hp at a 1:2 ratio, so 134.7 def is fixed by a94 hp, 190 hp is fixed by 136.7 def.

## Great League

...

### General Def/HP Azu

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
14,120,1496,46.0,91.873599,135.349498,196,0.0,13.0,14.0,2437268
12,948,1498,46.5,92.153599,137.407598,192,0.0,15.0,9.0,2431220
8,102,1496,46.0,91.873599,136.990098,193,0.0,15.0,11.0,2429054
11,44,1497,46.0,92.693899,133.708898,195,1.0,11.0,13.0,2416829
15,10,1497,47.5,92.713599,135.759198,192,0.0,12.0,7.0,2416650
21,241,1498,45.5,93.229199,134.936998,191,2.0,13.0,9.0,2402793
23,25,1492,45.0,92.944199,132.893898,194,2.0,11.0,13.0,2396233
18,192,1496,47.5,93.541399,134.931398,189,1.0,11.0,4.0,2385495
16,237,1495,49.0,93.553599,135.318598,188,0.0,10.0,1.0,2379993
17,161,1500,42.0,94.435400,133.650100,188,6.0,15.0,11.0,2372804


### Slight Atk Weight Azu

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
12,948,1498,46.5,92.153599,137.407598,192,0.0,15.0,9.0,2431220
11,44,1497,46.0,92.693899,133.708898,195,1.0,11.0,13.0,2416829
15,10,1497,47.5,92.713599,135.759198,192,0.0,12.0,7.0,2416650
21,241,1498,45.5,93.229199,134.936998,191,2.0,13.0,9.0,2402793
23,25,1492,45.0,92.944199,132.893898,194,2.0,11.0,13.0,2396233
18,192,1496,47.5,93.541399,134.931398,189,1.0,11.0,4.0,2385495
16,237,1495,49.0,93.553599,135.318598,188,0.0,10.0,1.0,2379993
17,161,1500,42.0,94.435400,133.650100,188,6.0,15.0,11.0,2372804
19,172,1492,48.0,93.823899,133.678298,189,1.0,9.0,3.0,2370479


## Ultra League

In [4]:
pvprankings.display_rs_info(df,'Deoxys (Defense)')

# Deoxys (Defense)s you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/deoxys-defense-pvp-iv-deep-dive-analysis)!

Check out the videos: [here](https://www.youtube.com/watch?v=p-UrAQDrvTQ)!

## Great League

### Best you can get are here, but watch the actual video

...

## Ultra League

### BB umbreon

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
8,1274,2491,49.0,132.812698,286.507897,122,15.0,13.0,10.0,4642330


### Registeel and TF

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [24]:
pvprankings.display_rs_info(df,'Venusaur')

# Venusaurs you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/venusaur-pvp-iv-deep-dive)!

## Great League

### Froslass Slayer + OK Def

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
5,1499,1499,20.5,123.384028,120.964733,122,6.0,11.0,12.0,1820864
18,267,1496,20.5,122.779204,120.359909,123,5.0,10.0,14.0,1817656
13,570,1488,20.5,123.384028,118.545438,122,6.0,7.0,13.0,1784446


...

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
2,2058,2499,38.0,160.460146,157.344415,154,8.0,13.0,8.0,3888116
8,934,2498,39.5,160.644616,158.282195,152,6.0,12.0,4.0,3864931


### Huge Defense

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### As good as Rank 1 Bulk

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,2498,2498,40.0,156.479400,159.640600,159,0.0,13.0,12.0,3971893
3,1829,2487,38.0,156.565483,158.902281,158,3.0,15.0,14.0,3930820


In [10]:
pvprankings.display_full_report(df)

# Jumpluffs you have, according to the Swag Man

## Great League

### Top 12 amazing stat product

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
25,1497,1497,43.5,95.320400,159.944400,154,0.0,15.0,10.0,2347878
17,329,1498,42.0,96.036000,158.459400,153,2.0,15.0,11.0,2328324
21,406,1495,44.0,95.615399,158.818798,153,0.0,13.0,9.0,2323384
3,80,1493,43.0,98.246600,158.644100,145,4.0,14.0,0.0,2260005


### Slight attack, high def

...

### Def

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
71,44,1496,19.0,124.025406,110.632991,131,5.0,15.0,1.0,1797490


### Atk

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
65,398,1497,19.0,121.696290,108.303875,139,1.0,11.0,15.0,1832044
13,396,1491,19.0,121.696290,108.886154,138,1.0,12.0,12.0,1828643
28,253,1491,19.0,121.696290,108.886154,138,1.0,12.0,12.0,1828643
7,215,1496,19.5,122.696685,109.129263,136,0.0,10.0,6.0,1821012
98,158,1500,19.5,123.286573,109.129263,135,1.0,10.0,5.0,1816313
14,222,1486,19.0,122.278569,109.468433,135,2.0,13.0,7.0,1807061
58,194,1496,19.0,124.025406,108.303875,134,5.0,11.0,6.0,1799945
71,44,1496,19.0,124.025406,110.632991,131,5.0,15.0,1.0,1797490
105,138,1483,18.5,124.106933,108.593566,131,8.0,14.0,4.0,1765515
49,386,1466,19.0,122.278569,108.886154,132,2.0,12.0,2.0,1757506


## Ultra League

In [121]:
pvprankings.display_rs_info(df,'Walrein')

Checking for ['Spheal', 'Sealeo', 'Walrein']


# Walreins you have, according to the Swag Man

Check out the article [here](https://gamepress.gg/pokemongo/walrein-pvp-iv-deep-dive#topic-372706)!

Check out the videos: [here](https://www.youtube.com/watch?v=yJUjtPAPkEM)!

## Great League

...

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### Mirror masters

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


### >147.75 and still ties most 0-0 mirrors

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
5,494,2494,38.0,147.997222,144.102559,197,8.0,9.0,11.0,4201375


In [71]:
imp.reload(pvprankings)
pvprankings.ivs_to_stats(5,10,9,10,mon='Jumpluff',max_level=51,max_cp=1500.99) # he says 2281559

(41.5,
 1499.1622806154755,
 98.12940184499999,
 153.975402895,
 151.58200284999998,
 2290330)

In [75]:
98.12940184499999*153.975402895*151.58200284999998, 98.12940184499999*153.975402895*151, int(98.12988234289728950444*153.97615684698521931750*151)

(2290330.4222420575, 2281536.6419243137, 2281558)

In [47]:
from IPython.display import display
from IPython.display import display, Markdown, Latex

In [49]:
display(Markdown('# Cows'))
display(pvprankings.get_rs_info(df,'Jumpluff')['ultra'])

# Cows

Checking for ['Hoppip', 'Skiploom', 'Jumpluff']


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,129,1690,51.0,102.281299,166.524098,163,3.0,14.0,13.0,2776265
1,337,1853,51.0,111.579599,166.524098,165,14.0,14.0,15.0,3065814
2,192,1713,51.0,104.817199,167.369398,159,6.0,15.0,8.0,2789367
3,154,1764,51.0,110.734299,158.071098,160,13.0,4.0,9.0,2800622
4,223,1790,51.0,111.579599,167.369398,153,14.0,15.0,1.0,2857276
5,56,1758,51.0,109.888999,167.369398,152,12.0,15.0,0.0,2795592
6,419,1659,51.0,101.435999,162.297598,164,2.0,9.0,14.0,2699902
7,406,1627,51.0,99.745399,165.678798,160,0.0,13.0,9.0,2644111
8,80,1646,51.0,103.126599,166.524098,152,4.0,14.0,0.0,2610305
9,179,1642,51.0,103.971899,162.297598,153,5.0,9.0,1.0,2581781


In [43]:
display(pvprankings.get_rs_info(df,'Jumpluff')['ultra'])

Checking for ['Hoppip', 'Skiploom', 'Jumpluff']


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,129,1690,51.0,102.281299,166.524098,163,3.0,14.0,13.0,2776265
1,337,1853,51.0,111.579599,166.524098,165,14.0,14.0,15.0,3065814
2,192,1713,51.0,104.817199,167.369398,159,6.0,15.0,8.0,2789367
3,154,1764,51.0,110.734299,158.071098,160,13.0,4.0,9.0,2800622
4,223,1790,51.0,111.579599,167.369398,153,14.0,15.0,1.0,2857276
5,56,1758,51.0,109.888999,167.369398,152,12.0,15.0,0.0,2795592
6,419,1659,51.0,101.435999,162.297598,164,2.0,9.0,14.0,2699902
7,406,1627,51.0,99.745399,165.678798,160,0.0,13.0,9.0,2644111
8,80,1646,51.0,103.126599,166.524098,152,4.0,14.0,0.0,2610305
9,179,1642,51.0,103.971899,162.297598,153,5.0,9.0,1.0,2581781


In [25]:
pvprankings.pokemon['Jumpluff']

{'dex': 189,
 'speciesName': 'Jumpluff',
 'speciesId': 'jumpluff',
 'baseStats': {'atk': 118, 'def': 183, 'hp': 181},
 'types': ['grass', 'flying'],
 'fastMoves': ['BULLET_SEED', 'INFESTATION'],
 'chargedMoves': ['DAZZLING_GLEAM', 'ENERGY_BALL', 'AERIAL_ACE', 'SOLAR_BEAM'],
 'defaultIVs': {'cp500': [11.5, 5, 15, 14],
  'cp1500': [41, 4, 10, 15],
  'cp2500': [50, 15, 15, 15],
  'cp1500l40': [40, 4, 13, 14]},
 'searchPriority': 5,
 'level25CP': 958,
 'tags': ['shadoweligible'],
 'buddyDistance': 3,
 'thirdMoveCost': 50000,
 'released': True}

In [22]:
import pandas as pd, numpy as np
import math

In [23]:
df = pd.read_csv('poke_genie_export.csv')

In [15]:
# You give a whole evolutionary line here.
mons = ['Spheal', 'Sealeo','Walrein']
mons = ['Bulbasaur', 'Ivysaur', 'Venusaur']
mons = ['Hoppip','Skiploom','Jumpluff']
#mons = ['Venusaur', 'Venusaur', 'Venusaur']
mon = mons[-1]
base_ivs = {
    # base_attack, base_defense, base_stamina
    'Walrein': (182, 176, 242),
    'Trevenant': (201,154,198),
    'Venusaur': (198, 189, 190),
    'Jumpluff': (118, 183, 181),
}[mon]
df = df[df.Name.isin(mons)]

In [16]:
df

,Index,Name,Form,Pokemon Number,Gender,CP,HP,Atk IV,Def IV,Sta IV,...,Sha/Pur (U),Rank % (L),Rank # (L),Stat Prod (L),Dust Cost (L),Candy Cost (L),Name (L),Form (L),Sha/Pur (L),Marked for PvP use
0,1,Hoppip,NaN,187,♂,129,49,3.0,14.0,13.0,...,0.0,95.38%,190.0,96.52%,242000.0,258.0,Hoppip,NaN,0.0,NaN
1,2,Hoppip,NaN,187,NaN,337,75,14.0,14.0,15.0,...,0.0,35.97%,2623.0,89.60%,70000.0,62.0,Hoppip,NaN,0.0,NaN
2,3,Hoppip,NaN,187,NaN,192,57,6.0,15.0,8.0,...,1.0,83.15%,691.0,94.97%,33364.0,67.0,Skiploom,NaN,1.0,NaN
3,4,Hoppip,NaN,187,NaN,154,50,13.0,4.0,9.0,...,0.0,17.61%,3375.0,87.78%,165000.0,160.0,Hoppip,NaN,0.0,NaN
4,5,Hoppip,NaN,187,NaN,223,55,14.0,15.0,1.0,...,0.0,13.77%,3532.0,87.37%,127600.0,116.0,Hoppip,NaN,0.0,NaN
5,6,Hoppip,NaN,187,NaN,56,28,12.0,15.0,0.0,...,0.0,29.87%,2873.0,89.00%,173400.0,172.0,Hoppip,NaN,0.0,NaN
6,7,Hoppip,NaN,187,NaN,419,91,2.0,9.0,14.0,...,0.0,91.62%,344.0,95.74%,181000.0,182.0,Hoppip,NaN,0.0,NaN
7,8,Hoppip,NaN,187,NaN,406,87,0.0,13.0,9.0,...,0.0,98.49%,63.0,97.61%,238000.0,182.0,Hoppip,NaN,0.0,NaN
8,9,Hoppip,NaN,187,NaN,80,35,4.0,14.0,0.0,...,0.0,81.29%,767.0,94.81%,46400.0,75.0,Skiploom,NaN,0.0,NaN
9,10,Hoppip,NaN,187,NaN,179,53,5.0,9.0,1.0,...,0.0,51.04%,2006.0,92.23%,32800.0,57.0,Skiploom,NaN,0.0,NaN


In [26]:
df.columns

Index(['Index', 'Name', 'Form', 'Pokemon Number', 'Gender', 'CP', 'HP',
       'Atk IV', 'Def IV', 'Sta IV', 'IV Avg', 'Level Min', 'Level Max',
       'Quick Move', 'Charge Move', 'Charge Move 2', 'Scan Date',
       'Original Scan Date', 'Catch Date', 'Weight', 'Height', 'Lucky',
       'Shadow/Purified', 'Favorite', 'Dust', 'Rank % (G)', 'Rank # (G)',
       'Stat Prod (G)', 'Dust Cost (G)', 'Candy Cost (G)', 'Name (G)',
       'Form (G)', 'Sha/Pur (G)', 'Rank % (U)', 'Rank # (U)', 'Stat Prod (U)',
       'Dust Cost (U)', 'Candy Cost (U)', 'Name (U)', 'Form (U)',
       'Sha/Pur (U)', 'Rank % (L)', 'Rank # (L)', 'Stat Prod (L)',
       'Dust Cost (L)', 'Candy Cost (L)', 'Name (L)', 'Form (L)',
       'Sha/Pur (L)', 'Marked for PvP use'],
      dtype='object')

In [27]:
# I definitely stole this table from https://gamepress.gg/pokemongo/cp-multiplier
def get_cpm(level):
    d = {
        1:	0.094,
        1.5: 0.1351374318,
        2: 0.16639787,
        2.5: 0.192650919,
        3: 0.21573247,
        3.5: 0.2365726613,
        4: 0.25572005,
        4.5: 0.2735303812,
        5: 0.29024988,
        5.5: 0.3060573775,
        6: 0.3210876,
        6.5: 0.3354450362,
        7: 0.34921268,
        7.5: 0.3624577511,
        8: 0.3752356,
        8.5: 0.387592416,
        9: 0.39956728,
        9.5: 0.4111935514,
        10: 0.4225,
        10.5: 0.4329264091,
        11: 0.44310755,
        11.5: 0.4530599591,
        12: 0.4627984,
        12.5: 0.472336093,
        13: 0.48168495,
        13.5: 0.4908558003,
        14: 0.49985844,
        14.5: 0.508701765,
        15: 0.51739395,
        15.5: 0.5259425113,
        16: 0.5343543,
        16.5: 0.5426357375,
        17: 0.5507927,
        17.5: 0.5588305862,
        18: 0.5667545,
        18.5: 0.5745691333,
        19: 0.5822789,
        19.5: 0.5898879072,
        20: 0.5974,
        20.5: 0.6048236651,
        21: 0.6121573,
        21.5: 0.6194041216,
        22: 0.6265671,
        22.5: 0.6336491432,
        23: 0.64065295,
        23.5: 0.6475809666,
        24: 0.65443563,
        24.5: 0.6612192524,
        25: 0.667934,
        25.5: 0.6745818959,
        26: 0.6811649,
        26.5: 0.6876849038,
        27: 0.69414365,
        27.5: 0.70054287,
        28: 0.7068842,
        28.5: 0.7131691091,
        29: 0.7193991,
        29.5: 0.7255756136,
        30: 0.7317,
        30.5: 0.7347410093,
        31: 0.7377695,
        31.5: 0.7407855938,
        32: 0.74378943,
        32.5: 0.7467812109,
        33: 0.74976104,
        33.5: 0.7527290867,
        34: 0.7556855,
        34.5: 0.7586303683,
        35: 0.76156384,
        35.5: 0.7644860647,
        36: 0.76739717,
        36.5: 0.7702972656,
        37: 0.7731865,
        37.5: 0.7760649616,
        38: 0.77893275,
        38.5: 0.7817900548,
        39: 0.784637,
        39.5: 0.7874736075,
        40: 0.7903,
        40.5: 0.792803968,
        41: 0.79530001,
        41.5: 0.797800015,
        42: 0.8003,
        42.5: 0.802799995,
        43: 0.8053,
        43.5: 0.8078,
        44: 0.81029999,
        44.5: 0.812799985,
        45: 0.81529999,
        45.5: 0.81779999,
        46: 0.82029999,
        46.5: 0.82279999,
        47: 0.82529999,
        47.5: 0.82779999,
        48: 0.83029999,
        48.5: 0.83279999,
        49: 0.83529999,
        49.5: 0.83779999,
        50: 0.84029999,
        50.5: 0.84279999,
        51: 0.84529999,        
        }
    return d[level]
get_cpm(2)

0.16639787

In [28]:
def ivs_to_stats(my_a, my_d, my_s, my_level, max_level=40,max_cp=1500.99, 
                 mon=mon, base_ivs=base_ivs):
    base_attack, base_defense, base_stamina = base_ivs
    attack = base_attack + my_a
    defense = my_d + base_defense
    stamina = my_s + base_stamina
    level = my_level#10
    cp = 10
    level_attack, level_defense, level_stamina, stat_prod = 0,0,0,0
    stats = (level, cp, level_attack, level_defense, level_stamina, stat_prod)
    while level <= max_level:
        cpm = get_cpm(level)
        cp = (attack * defense**0.5 * stamina**0.5 * cpm**2) / 10
        level_attack = attack * cpm
        level_defense = defense * cpm
        level_stamina = stamina * cpm
        if cp <= max_cp:
            stat_prod = math.floor(level_attack*level_defense*math.floor(level_stamina))
            stats = level, cp, level_attack, level_defense, level_stamina, stat_prod
        level = level + 0.5
    level, cp, level_attack, level_defense, level_stamina, stat_prod  = stats
    #print(f'{level_attack}, {level_defense}, {level_stamina}, {level_attack*level_defense*level_stamina}')
    return stats
    #print(f'level {level} cp {cp:.0f} attack {level_attack:.1f} defense {level_defense:.1f} stamina {level_stamina:.0f}')

In [29]:
ivs_to_stats(4,14,2,10)
ivs_to_stats(4,9,8,10)
ivs_to_stats(2,9,2,10)
ivs_to_stats(4,10,7,10)

(40.0, 1451.4476404036159, 96.4166, 152.5279, 148.5764, 2176520)

# Great League
## Note: max level is set to 40 in the line below. If you want to consider best buddies that aren't XL, set it to 41. If you want XL, set it to 50 or 51

In [37]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s, my_level = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV'], s['Level Min']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,my_level = my_level, max_level=51)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [38]:
# For each of these, you do this
def get_mons(attack,defense,hp,mine,*,level_max=99):    
    these = mine[mine.attack >= attack]
    these = these[these.defense >= defense]
    these = these[these.stamina >= hp]
    these = these[these.level <= level_max]
    these = these.sort_values('statprod',ascending=False)
    return these

In [39]:
# Walrein stats that meet the attack breakpoint
attack,defense,hp = 113.77, 113, 148
# Walrein stats that meet the GOD TIER
attack, defense, hp = 114.46,114.75, 148
# Walrein stats that hit all of the breakpoints
attack,defense,hp = 113.77, 113, 148
# Walrein great league high attack
attack, defense, hp = 114.46,113, 148
# Walrein great league waterfall
attack, defense, hp = 114.62, 113, 148

In [40]:
# Venusaur Frosslass Slayer OK def
# The 6/11/12 is a venu
attack, defense, hp = 122.5, 117.88, 122
# Venusaur Frosslass Slayer Big def
attack, defense, hp = 122.5, 121.13, 122
# Venusaur Big Bulk
attack, defense, hp = 0, 121.13, 123
# Worse version of that
attack, defense, hp = 0, 120.0, 123
# Venusaur shadow big bulk, you manually check if it's a shadow
attack, defense, hp = 0, 120.46, 122
# Venusaur GL Mega Frosslass, needs to be level 15 or maybe 15.5. check.
attack, defense, hp = 0, 0, 105
# Venusaur GL Mega Basti and Trev Atk
attack, defense, hp = 131.8, 0,0
#get_mons(attack, defense, hp, mine,level_max=15)

In [41]:
# Jumpluff, great league only
# Top 12 amazing stat product, yo
attack, defense, hp = 0, 157.31, 0
get_mons(attack, defense, hp, mine).sort_values('statprod',ascending=False)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
23,655,1497,43.5,95.320400,159.944400,154,0.0,15.0,10.0,2347878
25,377,1497,43.5,95.320400,159.944400,154,0.0,15.0,10.0,2347878
27,190,1498,42.0,96.036000,158.459400,153,2.0,15.0,11.0,2328324
7,406,1495,44.0,95.615399,158.818798,153,0.0,13.0,9.0,2323384
8,80,1493,43.0,98.246600,158.644100,145,4.0,14.0,0.0,2260005


## The rank 16 one I have is good enough
(0/15/10)

In [42]:
# Jumpluff Slight attack high def
attack, defense, hp = 97.6, 156.3, 0
get_mons(attack, defense, hp, mine).sort_values('statprod',ascending=False)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
2,192,1498,40.0,97.9972,156.4794,149,6.0,15.0,8.0,2284846
8,80,1493,43.0,98.2466,158.6441,145,4.0,14.0,0.0,2260005


In [43]:
# Slight attack ballance buff
attack, defense, hp = 97.6, 150, 151
get_mons(attack, defense, hp, mine).sort_values('statprod',ascending=False)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
11,401,1499,41.5,98.129402,153.975403,151,5.0,10.0,9.0,2281536


# UL requires recalculating
## Note: max level is set to 40 in the line below. If you want to consider best buddies that aren't XL, set it to 41. If you want XL, set it to 50 or 51

In [18]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s, my_level = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV'], s['Level Min']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,max_level=40,my_level=my_level,max_cp=2500.99)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [19]:
# Walrein UL supreme
attack, defense, hp = 147, 145.3, 197
# Walrein UL other ten
attack, defense, hp = 147.76, 143.89, 197
# Walrein UL hosing around
attack, defense, hp = 147.75, 145, 0


In [20]:
# Venusaur UL Gfisk Slayer
# The  6/12/4 is a shadow venu
attack, defense, hp = 159.2, 157.25, 152
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
2,2058,2499,38.0,160.460146,157.344415,154,8.0,13.0,8.0,3888116
8,934,2498,39.5,160.644616,158.282195,152,6.0,12.0,4.0,3864931


In [21]:
# Venusaur UL Huge Def
attack, defense, hp = 0,160.82, 156
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [22]:
# Venusaur UL As good as Rank 1 bulk
# The 2498 is already a venu
attack, defense, hp = 0, 158.05, 156
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,2498,2498,40.0,156.479400,159.640600,159,0.0,13.0,12.0,3971893
3,1829,2487,38.0,156.565483,158.902281,158,3.0,15.0,14.0,3930820


In [23]:
# Venusaur UL Shadow General Blend (you check for shadow)
attack, defense, hp = 161.59, 152.28, 153
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
6,1151,2495,35.0,162.213098,153.074332,154,15.0,12.0,13.0,3823921


# ML does not require recalculating because it's the raw numbers

In [24]:
# Walrein ML breakpoints
a,d,s = 15,14,13


In [25]:
# ML
#Want 15/14/13
print('Here are the ones that hit all of the BPs')
a,d,s = 15,14,13
these = mine[mine.a >= a]
these = these[these.d >= d]
these = these[these.s >= s]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
4,1546,2494,34.5,161.588268,154.001965,154,15.0,14.0,14.0,3832276


### Some stuff left over from halloween that needs to be converted to the current format

In [26]:
# gfisk attack bp
these = pht[pht.attack >= 167.98]
# hp breakpoint for mirror
these = these[these.stamina >= 167]
these

NameError: name 'pht' is not defined

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127.8]
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these # 3/15/7 is RS, I think maybe 3/14/11 for Azu

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127] 
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these

In [ ]:
ivs_to_stats(my_a=3,my_d=14,my_s=11,max_cp=1501)